## African influencers: Twitter users segmentation 
### The goal is to Identify influencers rank position from Twitter data.

##### Web scrape

In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys
import requests
import urllib.request
import time
import fire
import re

In [2]:
# Extracting data from website for 100 most influential Twitter users in Africa
page_url = 'https://africafreak.com/100-most-influential-twitter-users-in-africa'
page = requests.get(page_url)
soup = BeautifulSoup(page.content, 'html.parser')
data = soup.find(id='content-area')

In [3]:
#Filtering to find the name and username of the 100 most influential Twitter users in Africa
influencer_name = data.find_all('h2')
africa_influencers = []
for influencer in influencer_name:
    influencer = influencer.text
    africa_influencers.extend([influencer])
    
africa_influencers = [word.replace(".", "").replace("(", "").replace(")", "") for word in africa_influencers]
africa_influencers = africa_influencers[:-4]

In [4]:
#into a dataframe
africa_influencers = [influencer.split('@') for influencer in africa_influencers]
africa_influencers_data = pd.DataFrame(africa_influencers)


In [5]:
#Small Clean up
def remove(list): 
    pattern = '[0-9]'
    list = [re.sub(pattern, '', i) for i in list] 
    return list

In [6]:
#using the clean up
africa_influencers_data[0] = remove(africa_influencers_data[0])

In [7]:
#rename columns
africa_influencers_data = africa_influencers_data.rename(columns={0:'Name',1:'Username'})

In [8]:
#remove empty column
africa_influencers_data = africa_influencers_data.drop(columns = 2)

In [9]:
# Extracting data from website for African leaders respond to coronavirus… on Twitter
another_page_url = 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
another_page = requests.get(another_page_url)
another_soup = BeautifulSoup(another_page.content, 'html.parser')
more_data = another_soup.find(id="content")

In [10]:
#Filtering to find the name and username for African leaders who responded to coronavirus… on Twitter
leader_name = more_data.find_all('blockquote')
splitting = []
for leader in leader_name:
    leader = leader.text
    leader = leader.split('—')
    splitting.extend([leader[-1]])

In [11]:
african_leaders = []
for leader in splitting:
    leader = leader.split('March')
    african_leaders.extend([leader[0]])

In [12]:
#Small Clean up
def special_remove(list): 
    list = [word.replace("(", "").replace(")", "")for word in list]
    return list

In [13]:
#using cleanup
african_leaders = special_remove(african_leaders)
#the clean got rid of numbers 

In [14]:
#to dataframe
african_leaders = [leader.split('@') for leader in african_leaders]
african_leaders_data = pd.DataFrame(african_leaders)

In [15]:
african_leaders_data = african_leaders_data.rename(columns={0:'Name',1:'Username'})

In [16]:
leader_usernames = list(african_leaders_data['Username'])
influencer_usernames = list(africa_influencers_data['Username'])
leader_usernames = [username.replace(" ", "") for username in leader_usernames]

In [17]:
pd.DataFrame(leader_usernames).to_csv('data/leader_usernames.csv')
pd.DataFrame(influencer_usernames).to_csv('data/influencer_usernames.csv')

##### Use twitter data to rank the top leaders and influencers.

In [18]:
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
from collections import Counter
from datetime import datetime, date, time, timedelta
import twitter


In [19]:
#Import the necessary methods from tweepy library  
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

#sentiment analysis package
from textblob import TextBlob

#general text pre-processor
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

#tweet pre-processor 
import preprocessor as p

from dotenv import load_dotenv
load_dotenv()

from pathlib import Path  # Python 3.6+ only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smwik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smwik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
#Variables that contains the user credentials to access Twitter API 
consumer_key = os.environ.get('TWITTER_API_KEY')
consumer_secret = os.environ.get('TWITTER_API_SECRET')
access_token = os.environ.get('TWITTER_ACCESS_TOKEN')
access_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')

#This handles Twitter authetification and the connection to Twitter Streaming API
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


###### Getting the reach_score

In [21]:
#find user object to get user data
african_leaders_dataset = []
for username in leader_usernames:
    try:
        search = api.search_users(username) 
        for user in search: 
            if user.screen_name == username:
                africa_leaders_data = []
                africa_leaders_data.extend([user.name,user.screen_name,user.followers_count,user.friends_count,user.statuses_count])
                african_leaders_dataset.extend([africa_leaders_data])
                
    except:
        continue

In [22]:
african_influencers_dataset = []
for username in influencer_usernames:
    try:
        search = api.search_users(username) 
        for user in search: 
            if user.screen_name == username:
                africa_influencers_data = []
                africa_influencers_data.extend([user.name,user.screen_name,user.followers_count,user.friends_count,user.statuses_count])
                african_influencers_dataset.extend([africa_influencers_data])
    except:
        continue

In [23]:
african_influencers_dataframe = pd.DataFrame(african_influencers_dataset,columns = ['Name','Username','Number of Followers','Number of Following','Number of Retweets and Tweets'])
african_leaders_dataframe = pd.DataFrame(african_leaders_dataset,columns = ['Name','Username','Number of Followers','Number of Following','Number of Retweets and Tweets'])

#getting their reach_score = followers-following
african_influencers_dataframe['reach_score']= african_influencers_dataframe['Number of Followers'] - african_influencers_dataframe['Number of Following']
african_leaders_dataframe['reach_score']= african_leaders_dataframe['Number of Followers'] - african_leaders_dataframe['Number of Following']

african_leaders_dataframe.to_csv('data/leaders_reach.csv', index = False, header=True)
african_influencers_dataframe.to_csv('data/influencers_reach.csv', index = False, header=True)

In [24]:
top_reach_influencers = african_influencers_dataframe.sort_values('reach_score',ascending=False).head(10)
top_reach_influencers.to_csv ('data/top_reach_influencers.csv', index = False, header=True)

top_reach_leaders = african_leaders_dataframe.sort_values('reach_score',ascending=False).head(10)
top_reach_leaders.to_csv ('data/top_reach_leaders.csv', index = False, header=True)

##### Find tweets

In [25]:
class tweetsearch():
    '''
    This is a basic class to search and download twitter data.
    You can build up on it to extend the functionalities for more 
    sophisticated analysis
    '''
    def __init__(self, cols=None,auth=None):
        #
        if not cols is None:
            self.cols = cols
        else:
            self.cols = ['id', 'created_at', 'source', 'original_text','clean_text', 
                    'sentiment','polarity','subjectivity', 'lang',
                    'favorite_count', 'retweet_count', 'original_author',   
                    'possibly_sensitive', 'hashtags',
                    'user_mentions', 'place', 'place_coord_boundaries']
            
        if auth is None:
            
            #Variables that contains the user credentials to access Twitter API 
            consumer_key = os.environ.get('TWITTER_API_KEY')
            consumer_secret = os.environ.get('TWITTER_API_SECRET')
            access_token = os.environ.get('TWITTER_ACCESS_TOKEN')
            access_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')
            


            #This handles Twitter authetification and the connection to Twitter Streaming API
            auth = OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_token_secret)
            

        #            
        self.auth = auth
        self.api = tweepy.API(auth,wait_on_rate_limit=True) 
        self.filtered_tweet = ''
            

    def clean_tweets(self, twitter_text):

        #use pre processor
        tweet = p.clean(twitter_text)

         #HappyEmoticons
        emoticons_happy = set([
            ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
            ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
            '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
            'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
            '<3'
            ])

        # Sad Emoticons
        emoticons_sad = set([
            ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
            ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
            ':c', ':{', '>:\\', ';('
            ])

        #Emoji patterns
        emoji_pattern = re.compile("["
                 u"\U0001F600-\U0001F64F"  # emoticons
                 u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                 u"\U0001F680-\U0001F6FF"  # transport & map symbols
                 u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                 u"\U00002702-\U000027B0"
                 u"\U000024C2-\U0001F251"
                 "]+", flags=re.UNICODE)

        #combine sad and happy emoticons
        emoticons = emoticons_happy.union(emoticons_sad)

        stop_words = set(stopwords.words('english'))
        word_tokens = nltk.word_tokenize(tweet)
        #after tweepy preprocessing the colon symbol left remain after      
        #removing mentions
        tweet = re.sub(r':', '', tweet)
        tweet = re.sub(r'‚Ä¶', '', tweet)

        #replace consecutive non-ASCII characters with a space
        tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)

        #remove emojis from tweet
        tweet = emoji_pattern.sub(r'', tweet)

        #filter using NLTK library append it to a string
        filtered_tweet = [w for w in word_tokens if not w in stop_words]

        #looping through conditions
        filtered_tweet = []    
        for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
            if w not in stop_words and w not in emoticons and w not in string.punctuation:
                filtered_tweet.append(w)

        return ' '.join(filtered_tweet)            

    def get_tweets(self, keyword, csvfile=None):
        
        
        df = pd.DataFrame(columns=self.cols)
        

        #page attribute in tweepy.cursor and iteration
        for page in tweepy.Cursor(self.api.search, q=keyword,count=20, include_rts=False,tweet_mode='extended' , since = '2020-05-01').pages(5):

            # the you receive from the Twitter API is in a JSON format and has quite an amount of information attached
            for status in page:
                
                new_entry = []
                status = status._json
                
                #if this tweet is a retweet update retweet count
                if status['created_at'] in df['created_at'].values:
                    i = df.loc[df['created_at'] == status['created_at']].index[0]
                    #
                    cond1 = status['favorite_count'] != df.at[i, 'favorite_count']
                    cond2 = status['retweet_count'] != df.at[i, 'retweet_count']
                    if cond1 or cond2:
                        df.at[i, 'favorite_count'] = status['favorite_count']
                        df.at[i, 'retweet_count'] = status['retweet_count']
                    continue

                #calculate sentiment
                filtered_tweet = self.clean_tweets(status['full_text'])
                blob = TextBlob(filtered_tweet)
                Sentiment = blob.sentiment     
                polarity = Sentiment.polarity
                subjectivity = Sentiment.subjectivity

                new_entry += [status['id'], status['created_at'],
                              status['source'], status['full_text'], filtered_tweet, 
                              Sentiment,polarity,subjectivity, status['lang'],
                              status['favorite_count'], status['retweet_count']]

                new_entry.append(status['user']['screen_name'])

                try:
                    is_sensitive = status['possibly_sensitive']
                except KeyError:
                    is_sensitive = None

                new_entry.append(is_sensitive)

                hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
                new_entry.append(hashtags) #append the hashtags

                #
                mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
                new_entry.append(mentions) #append the user mentions

                try:
                    xyz = status['place']['bounding_box']['coordinates']
                    coordinates = [coord for loc in xyz for coord in loc]
                except TypeError:
                    coordinates = None
                #
                new_entry.append(coordinates)

                try:
                    location = status['user']['location']
                except TypeError:
                    location = ''
                #
                new_entry.append(location)

                #now append a row to the dataframe
                single_tweet_df = pd.DataFrame([new_entry], columns=self.cols)
                df = df.append(single_tweet_df, ignore_index=True)

        #
        df['timestamp'] = df.created_at.map(pd.Timestamp)
        df = df.sort_values('timestamp').set_index('timestamp')
        df = df.drop('id',axis=1)
        
        if not csvfile is None:
            #save it to file
            df.to_csv(csvfile,mode='a', index=True, encoding="utf-8")
            

        return df 
    
    def get_tweets_again(self, username, csvfile=None):
        
        
        df = pd.DataFrame(columns=self.cols)
        

        #page attribute in tweepy.cursor and iteration
        for page in tweepy.Cursor(self.api.user_timeline, screen_name=username, count=20, include_rts=False,tweet_mode='extended', since = '2020-05-01').pages(5):

            # the you receive from the Twitter API is in a JSON format and has quite an amount of information attached
            for status in page:
                
                new_entry = []
                status = status._json
                
                #if this tweet is a retweet update retweet count
                if status['created_at'] in df['created_at'].values:
                    i = df.loc[df['created_at'] == status['created_at']].index[0]
                    #
                    cond1 = status['favorite_count'] != df.at[i, 'favorite_count']
                    cond2 = status['retweet_count'] != df.at[i, 'retweet_count']
                    if cond1 or cond2:
                        df.at[i, 'favorite_count'] = status['favorite_count']
                        df.at[i, 'retweet_count'] = status['retweet_count']
                    continue

                #calculate sentiment
                filtered_tweet = self.clean_tweets(status['full_text'])
                blob = TextBlob(filtered_tweet)
                Sentiment = blob.sentiment     
                polarity = Sentiment.polarity
                subjectivity = Sentiment.subjectivity

                new_entry += [status['id'], status['created_at'],
                              status['source'], status['full_text'], filtered_tweet, 
                              Sentiment,polarity,subjectivity, status['lang'],
                              status['favorite_count'], status['retweet_count']]

                new_entry.append(status['user']['screen_name'])

                try:
                    is_sensitive = status['possibly_sensitive']
                except KeyError:
                    is_sensitive = None

                new_entry.append(is_sensitive)

                hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
                new_entry.append(hashtags) #append the hashtags

                #
                mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
                new_entry.append(mentions) #append the user mentions

                try:
                    xyz = status['place']['bounding_box']['coordinates']
                    coordinates = [coord for loc in xyz for coord in loc]
                except TypeError:
                    coordinates = None
                #
                new_entry.append(coordinates)

                try:
                    location = status['user']['location']
                except TypeError:
                    location = ''
                #
                new_entry.append(location)

                #now append a row to the dataframe
                single_tweet_df = pd.DataFrame([new_entry], columns=self.cols)
                df = df.append(single_tweet_df, ignore_index=True)

        #
        df['timestamp'] = df.created_at.map(pd.Timestamp)
        df = df.sort_values('timestamp').set_index('timestamp')
        df = df.drop('id',axis=1)

        if not csvfile is None:
            #save it to file
            df.to_csv(csvfile, index=True,mode='a', encoding="utf-8")
            

        return df 

In [26]:
for username in leader_usernames:
    try:
        ts = tweetsearch()
        df = ts.get_tweets_again(username, csvfile='data/Leaders_Tweets_User.csv')
        #Helps keep track of usernames
        print(username)
    except tweepy.TweepError:
        continue

EswatiniGovern1
MalawiGovt
hagegeingob
FinanceSC
PresidencyZA
mohzambia
edmnangagwa
MinSantedj
hawelti
StateHouseKenya
PaulKagame
M_Farmaajo
SouthSudanGov
SudanPMHamdok
TZSpokesperson
KagutaMuseveni
angola_Mirex
willynyamitwe
Cherif_MZ
Presidence_RDC
PresidentABO
PresidenceBenin
rochkaborepf
PresidenciaCV
AOuattara_PRCI
Presidency_GMB
NAkufoAddo
President_GN
USEmbalo
PresidenceMali
CheikhGhazouani
IssoufouMhm
MBuhari
Macky_Sall
PresidentBio
MSPS_Togo


In [27]:
for username in leader_usernames:
    try:
        ts = tweetsearch()
        df = ts.get_tweets(username, csvfile='data/Leaders_Tweets.csv')
        #Helps keep track of usernames
        print(username)
    except tweepy.TweepError:
        continue


EswatiniGovern1
MalawiGovt
hagegeingob
FinanceSC
PresidencyZA
mohzambia
edmnangagwa
MinSantedj
hawelti
StateHouseKenya
PaulKagame
M_Farmaajo
SouthSudanGov
SudanPMHamdok
TZSpokesperson
KagutaMuseveni
angola_Mirex
willynyamitwe
Cherif_MZ
Presidence_RDC
PresidentABO
PresidenceBenin
rochkaborepf
PresidenciaCV
AOuattara_PRCI
Presidency_GMB
NAkufoAddo
President_GN
USEmbalo
PresidenceMali
CheikhGhazouani
IssoufouMhm
MBuhari
Macky_Sall
PresidentBio
MSPS_Togo


In [28]:
for username in influencer_usernames:
    try:
        ts = tweetsearch()
        df = ts.get_tweets_again(username, csvfile='data/Influencers_Tweets_User.csv')
        #Helps keep track of usernames
        print(username)
    except tweepy.TweepError:
        continue


gettleman
a24media
andiMakinana
AfricaCheck
JamesCopnall
oafrica
PatrickNgowi
StateAfrica
Moadow
BrendanSAfrica
CityTshwane
VISI_Mag
ThisIsAfricaTIA
sarzss
TheEIU_Africa
InvestInAfrica
malonebarry
artsouthafrica
KahnMorbee
JamalMOsman
iamsuede
mikestopforth
equal_education
t_mcconnell
forbeesta
jaxpanik
thisisafrica
audisouthafrica
ONEinAfrica
Hamza_Africa
africatechie
cx73
ayittey
MercedesBenz_SA
africagathering
okayafrica
mary_harper
savetherhino
pressfreedom 
TechCentral
GautengProvince
Aynte
daniel_howden
rangerdiaries
TheStar_news
schneiderhome
theafricareport
CityofJoburgZA
ThinkAfricaFeed
AfricaGoodNews
willintune
cnbcafrica
MadeItInAfrica
AfricaResearch
FoodBlogCT
MbuyiseniNdlozi
africaprogress
IFCAfrica
HenleyAfrica
geoffreyyork
Entrepreneur_SA
forbesafrica
IECSouthAfrica
art2gee
JendayiFrazer
PeterGreste
NDOCKenya
Mo_IbrahimFdn
ParliamentofRSA
SandtonCity
_AfricanUnion
gertjohan
SmithInAfrica
hartleyr
liveamp
SamsungSA
BobSkinstad
Camfed
BBCAndrewH
euphonik
UlrichJvV
Official

In [29]:
for username in influencer_usernames:
    try:
        ts = tweetsearch()
        df = ts.get_tweets(username, csvfile='data/Influencers_Tweets.csv')
        #Helps keep track of usernames
        print(username)
    except tweepy.TweepError:
        continue


gettleman
a24media
andiMakinana
AfricaCheck
JamesCopnall
oafrica
PatrickNgowi
StateAfrica
Moadow
BrendanSAfrica
CityTshwane
VISI_Mag
andBeyondSafari
ThisIsAfricaTIA
sarzss
TheEIU_Africa
InvestInAfrica
malonebarry
artsouthafrica
KahnMorbee
JamalMOsman
iamsuede
mikestopforth
equal_education
t_mcconnell
forbeesta
hurricanevaness
BBCKarenAllen
jaxpanik
thisisafrica
audisouthafrica
ONEinAfrica
Hamza_Africa
drewfhinshaw
africatechie
cx73
ayittey
MercedesBenz_SA
africagathering
okayafrica
mary_harper
savetherhino
pressfreedom 
TechCentral
GautengProvince
Aynte
daniel_howden
rangerdiaries
TheStar_news
schneiderhome
Afrinnovator
theafricareport
CityofJoburgZA
ThinkAfricaFeed
AfricaGoodNews
willintune
cnbcafrica
MadeItInAfrica
AfricaResearch
FoodBlogCT
MbuyiseniNdlozi
africaprogress
IFCAfrica
HenleyAfrica
The_New_Age
geoffreyyork
Entrepreneur_SA
forbesafrica
IECSouthAfrica
art2gee
JendayiFrazer
PeterGreste
NDOCKenya
Mo_IbrahimFdn
ParliamentofRSA
SandtonCity
_AfricanUnion
gertjohan
SmithInAfrica


In [30]:
# Want to stream it so my information is always updated
class StdOutListener(StreamListener):

    def __init__(self,fhandle, stop_at = 100):
        self.tweet_counter = 0
        self.stop_at = stop_at
        self.fhandle = fhandle
         
        
    def on_data(self, data):
        self.fhandle.write(data)
        
        #stop if enough tweets are obtained
        self.tweet_counter += 1   
        if self.tweet_counter < self.stop_at:        
            return True
        else:
            print('Max number of tweets reached: #tweets = ' + str(self.tweet_counter))
            return False

    def on_error(self, status):
        print (status)

def stream_tweet_data(filename='data/tweets_data.csv',
                      keywords=username,
                      is_async=False):
    # tweet topics to use as a filter. The tweets downloaded
    # will have one of the topics in their text or hashtag 

    print('saving data to file: ',filename)

    #print the tweet topics 
    print('Tweet Users are: ',keywords)

    #Variables that contains the user credentials to access Twitter API 
    consumer_key = os.environ.get('TWITTER_API_KEY')
    consumer_secret = os.environ.get('TWITTER_API_SECRET')
    access_token = os.environ.get('TWITTER_ACCESS_TOKEN')
    access_token_secret = os.environ.get('TWITTER_ACCESS_TOKEN_SECRET')
    

    #open file 
    fhandle=open(filename,'a+')

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener(fhandle)
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: first argument to this code
    stream.filter(track=keywords,is_async=is_async)

    return None




In [31]:
Leaders_Tweets = 'data/Streamed_Leaders_Tweets.json'
Influencers_Tweets = 'data/Streamed_Influencers_Tweets.json'

In [32]:
#warning, May take longer than expected
for username in leader_usernames:
    try:
        stream_tweet_data(filename=Leaders_Tweets, keywords = username)    
    except:
        continue
        

saving data to file:  data/Streamed_Leaders_Tweets.json
Tweet Users are:  EswatiniGovern1
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Leaders_Tweets.json
Tweet Users are:  MalawiGovt
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Leaders_Tweets.json
Tweet Users are:  hagegeingob
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Leaders_Tweets.json
Tweet Users are:  FinanceSC
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Leaders_Tweets.json
Tweet Users are:  PresidencyZA
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Leaders_Tweets.json
Tweet Users are:  mohzambia
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Leaders_Tweets.json
Tweet Users are:  edmnangagwa
420
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Leaders_Tweets.json
Tweet Users are:  MinSantedj
M

In [ ]:
for username in influencer_usernames:
    try:
        stream_tweet_data(filename=Influencers_Tweets, keywords = username)    
    except:
        continue

saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  gettleman
420
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  a24media
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  andiMakinana
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  AfricaCheck
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  JamesCopnall
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  oafrica
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  PatrickNgowi
420
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json


Tweet Users are:  MbuyiseniNdlozi
420
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  africaprogress
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  IFCAfrica
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  HenleyAfrica
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  The_New_Age
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  geoffreyyork
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  Entrepreneur_SA
420
Max number of tweets reached: #tweets = 100
saving data to file:  data/Streamed_Influencers_Tweets.json
Tweet Users are:  forbesafrica
Max number o